# Loading data from colorhunt

In [4]:
from urllib import request,parse
import json
import time
import random
from tqdm import trange

def load():
    data = []
    for i in trange(0,64,desc='Loading data'):
        body = parse.urlencode({'step':i,'sort':'new','tags':'','timeframe':30}).encode()
        Req = request.Request('https://colorhunt.co/php/feed.php',method='POST',headers={'content-type': 'application/x-www-form-urlencoded','User-Agent' :'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'})
        readed = request.urlopen(Req,data=body).read()
        loads = json.loads(readed)
        data += loads
        time.sleep(random.randint(5,10)/100) # slow down to prevent http 429
    return data

with open('dataset/color.json','w') as f:
    f.write(json.dumps(load()))

Loading data: 100%|██████████| 64/64 [00:32<00:00,  1.98it/s]


# Process data



In [10]:
import json
import time
from tqdm import trange
import random

def color_code_split(code):
    return [f'#{code[i:i+6]}'.upper() for i in range(0,len(code),6)]

def hex_2_rbg(hex):
    hex = hex.replace('##','#')
    r = str(int('0x'+ hex[1:3],16))
    g = str(int('0x'+ hex[3:5],16))
    b = str(int('0x'+ hex[5:],16))
    return (r,g,b)

raw_data = []

with open('dataset/color.json','r') as f:
    raw_data = json.loads(f.read())

data = []
for i in trange(len(raw_data),desc="Transform color codes"):
    hex = color_code_split(raw_data[i]['code']) # transform color_code in data to hex
    for l in range(len(hex)):
        row = [str(i),'|'.join(hex_2_rbg(hex[l]))] # create palette id
        row.append(raw_data[i]['likes'])
        data.append(row)
    time.sleep(0.0001)

with open('dataset/color_processed.csv','w') as f:
    f.write("paletteId,color_hex,likes\n" + '\n'.join([','.join(i) for i in data]))

Transform color codes: 100%|██████████| 2546/2546 [00:39<00:00, 64.65it/s]


# Training

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


df = pd.read_csv('dataset/color_processed.csv')
print(df['color_hex'])

0           26|26|64
1           39|0|130
2         122|11|192
3         250|88|182
4        249|228|212
            ...     
10179       34|14|36
10180    125|216|125
10181     76|145|115
10182      91|68|106
10183     144|99|135
Name: color_hex, Length: 10184, dtype: object
